# TP Color 

## Importation des Bibliothèques

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, UpSampling2D, Input

## Différents Encodage de Couleurs (Négatif,,RGB,Noir et Blanc)

In [ ]:
image = cv2.imread('Image_Color.jpg')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
print(image.shape)
print(image_gray.shape)
fig=plt.figure(figsize=(15, 8))
fig.add_subplot(1,3, 1)
plt.title('Encodage BRG (natif cv2) affiché en RGB')
plt.imshow(image)
fig.add_subplot(1,3, 2)
plt.title('Encodage RGB')
plt.imshow(image_rgb)
fig.add_subplot(1,3, 3)
plt.title('Encodage niveau de gris')
plt.imshow(image_gray,cmap='gray')

## Séparation des Couches L (Lightness), a et b 

In [ ]:
image_Lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
print(image_Lab.shape)
fig=plt.figure(figsize=(15, 8))
## Affichage L (on repasse en RGB noté les valeur des couches a et b à 128)
fig.add_subplot(1,3, 1)
plt.title('couche L')
image_L = image_Lab.copy()
image_L[:,:,1:]=128
image_L = cv2.cvtColor(image_L, cv2.COLOR_LAB2RGB)
plt.imshow(image_L)
## Affichage a (on repasse en RGB noté les valeurs L et b à 128)
fig.add_subplot(1,3, 2)
plt.title('Couche a')
image_a = image_Lab.copy()
image_a[:,:,0]=128
image_a[:,:,2]=128
image_a = cv2.cvtColor(image_a, cv2.COLOR_LAB2RGB)
plt.imshow(image_a)
## Affichage a (on repasse en RGB noté les valeurs L et a à 128)
fig.add_subplot(1,3, 3)
plt.title('Couche b')
image_b = image_Lab.copy()
image_b[:,:,0]=128
image_b[:,:,1]=128
image_b = cv2.cvtColor(image_b, cv2.COLOR_LAB2RGB)
plt.imshow(image_b)

## Colorisation Automatique d'Images en Niveaux de Gris avec un Réseau Encodeur-Décodeur

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, MaxPooling2D, Concatenate
from tensorflow.keras.datasets import cifar10

# Charger les données (CIFAR-10 comme exemple, convertir en niveaux de gris)
(x_train, _), (x_test, _) = cifar10.load_data()

# Normaliser les données
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convertir en niveaux de gris
def rgb_to_grayscale(images):
    return np.dot(images[..., :3], [0.2989, 0.5870, 0.1140])[..., np.newaxis]

gray_train = rgb_to_grayscale(x_train)
gray_test = rgb_to_grayscale(x_test)

# Construire le réseau encodeur-décodeur
input_img = Input(shape=(32, 32, 1))

# Encodeur
x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

# Décodeur
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
out_img = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Modèle
model = Model(input_img, out_img)
model.compile(optimizer='adam', loss='mse')

# Entraîner le modèle
model.fit(gray_train, x_train, epochs=10, batch_size=128, validation_split=0.1)

# Tester le modèle
predicted_color = model.predict(gray_test[:10])

# Fonction pour afficher les résultats
def plot_images(original_gray, predicted_color, original_color):
    plt.figure(figsize=(15, 5))
    for i in range(10):
        # Image en niveaux de gris
        plt.subplot(3, 10, i + 1)
        plt.imshow(original_gray[i].squeeze(), cmap='gray')
        plt.axis('off')

        # Image colorisée prédite
        plt.subplot(3, 10, i + 11)
        plt.imshow(predicted_color[i])
        plt.axis('off')

        # Image couleur originale
        plt.subplot(3, 10, i + 21)
        plt.imshow(original_color[i])
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Afficher les résultats
plot_images(gray_test[:10], predicted_color, x_test[:10])
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, MaxPooling2D, Concatenate
from tensorflow.keras.datasets import cifar10

# Charger les données (CIFAR-10 comme exemple, convertir en niveaux de gris)
(x_train, _), (x_test, _) = cifar10.load_data()

# Normaliser les données
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convertir en niveaux de gris
def rgb_to_grayscale(images):
    return np.dot(images[..., :3], [0.2989, 0.5870, 0.1140])[..., np.newaxis]

gray_train = rgb_to_grayscale(x_train)
gray_test = rgb_to_grayscale(x_test)

# Construire le réseau encodeur-décodeur
input_img = Input(shape=(32, 32, 1))

# Encodeur
x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

# Décodeur
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
out_img = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Modèle
model = Model(input_img, out_img)
model.compile(optimizer='adam', loss='mse')

# Entraîner le modèle
model.fit(gray_train, x_train, epochs=10, batch_size=128, validation_split=0.1)

# Tester le modèle
predicted_color = model.predict(gray_test[:10])

# Fonction pour afficher les résultats
def plot_images(original_gray, predicted_color, original_color):
    plt.figure(figsize=(15, 5))
    for i in range(10):
        # Image en niveaux de gris
        plt.subplot(3, 10, i + 1)
        plt.imshow(original_gray[i].squeeze(), cmap='gray')
        plt.axis('off')

        # Image colorisée prédite
        plt.subplot(3, 10, i + 11)
        plt.imshow(predicted_color[i])
        plt.axis('off')

        # Image couleur originale
        plt.subplot(3, 10, i + 21)
        plt.imshow(original_color[i])
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Afficher les résultats
plot_images(gray_test[:10], predicted_color, x_test[:10])

## Construction et Entraînement d'un Réseau Encodeur-Décodeur pour la Reconstruction d'Images en Niveaux de Gris

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, MaxPooling2D, Concatenate, BatchNormalization
from tensorflow.keras.datasets import cifar10, mnist, fashion_mnist
from sklearn.model_selection import train_test_split
import gc
import cv2

# Fonction pour charger et normaliser les données de plusieurs bases de données
def load_and_prepare_datasets():
    datasets = []
    
    # CIFAR-10
    (x_train, _), (x_test, _) = cifar10.load_data()
    datasets.append((x_train, x_test))
    
    # MNIST (redimensionnement en 32x32)
    (x_train, _), (x_test, _) = mnist.load_data()
    x_train = np.expand_dims(x_train, axis=-1)
    x_test = np.expand_dims(x_test, axis=-1)
    x_train = np.tile(x_train, (1, 1, 1, 3))
    x_test = np.tile(x_test, (1, 1, 1, 3))
    x_train = np.array([cv2.resize(img, (32, 32)) for img in x_train])
    x_test = np.array([cv2.resize(img, (32, 32)) for img in x_test])
    datasets.append((x_train, x_test))
    
    # Fashion MNIST (redimensionnement en 32x32)
    (x_train, _), (x_test, _) = fashion_mnist.load_data()
    x_train = np.expand_dims(x_train, axis=-1)
    x_test = np.expand_dims(x_test, axis=-1)
    x_train = np.tile(x_train, (1, 1, 1, 3))
    x_test = np.tile(x_test, (1, 1, 1, 3))
    x_train = np.array([cv2.resize(img, (32, 32)) for img in x_train])
    x_test = np.array([cv2.resize(img, (32, 32)) for img in x_test])
    datasets.append((x_train, x_test))
    
    return datasets

# Convertir les images en niveaux de gris
def rgb_to_grayscale(images):
    return np.dot(images[..., :3], [0.2989, 0.5870, 0.1140])[..., np.newaxis]

# Construire le réseau encodeur-décodeur
def build_model(input_shape=(32, 32, 1)):
    input_img = Input(shape=input_shape)
    
    # Encodeur
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    
    # Décodeur
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    out_img = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
    
    model = Model(input_img, out_img)
    model.compile(optimizer='adam', loss='mse')
    return model

# Fonction pour entraîner le modèle sur plusieurs bases de données
def train_on_multiple_datasets(model, datasets, epochs=10, batch_size=128):
    history = {'loss': [], 'val_loss': []}
    for i, (x_train, x_test) in enumerate(datasets):
        x_train = x_train.astype('float32') / 255.0
        x_test = x_test.astype('float32') / 255.0
        
        gray_train = rgb_to_grayscale(x_train)
        gray_test = rgb_to_grayscale(x_test)
        
        history_part = model.fit(gray_train, x_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)
        history['loss'].extend(history_part.history['loss'])
        history['val_loss'].extend(history_part.history['val_loss'])
        
        # Libérer l'espace mémoire
        del x_train, x_test, gray_train, gray_test, history_part
        gc.collect()
    
    return history

# Fonction pour afficher les courbes d'apprentissage
def plot_learning_curves(history):
    plt.figure(figsize=(10, 5))
    plt.plot(history['loss'], label='Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Courbes d’apprentissage')
    plt.legend()
    plt.show()

# Fonction pour calculer le MSE score
def compute_mse_score(model, x_test):
    x_test = x_test.astype('float32') / 255.0
    gray_test = rgb_to_grayscale(x_test)
    predicted_color = model.predict(gray_test)
    mse_score = np.mean((predicted_color - x_test) ** 2)
    return mse_score

# Chargement des datasets
all_datasets = load_and_prepare_datasets()

# Construction du modèle
model = build_model()

# Entraînement du modèle sur toutes les bases de données
history = train_on_multiple_datasets(model, all_datasets, epochs=10, batch_size=128)

# Affichage des courbes d'apprentissage
plot_learning_curves(history)

# Calcul du MSE score
(x_train, x_test) = all_datasets[0]
mse_score = compute_mse_score(model, x_test)
print(f'MSE Score: {mse_score}')

# Libérer l'espace mémoire
for dataset in all_datasets:
    del dataset
    gc.collect()

# Tester le modèle et afficher les résultats
x_test = x_test.astype('float32') / 255.0
gray_test = rgb_to_grayscale(x_test)
predicted_color = model.predict(gray_test[:10])

# Libérer l'espace mémoire
del x_test, gray_test
gc.collect()

plot_images(gray_test[:10], predicted_color, x_test[:10])